![](https://i.imgur.com/62lgQpf.jpg)

-----

## About the dataset
https://www.kaggle.com/nowke9/ipldata
- `teams.csv`: list of IPL teams
- `matches.csv`: IPL match data
- `deliveries.csv`: Ball by ball data of IPL matches.

## Questions we are gonna answer...

- Which team has taken maximum number of correct toss decisions
- Which team has won/lost maximum number of matches while chasing?
- Which toss decision is best for each ground?
- Which player has won maximum number of MoM awards while chasing?

- Which batsman has played maximum number of dot balls?
- Which batsmen came to crease but never got a chance to face a ball?
- Which bowler has hit for maximum number of 1s/2s/3s/4s/6s?
- Which batsman got stumped out maximum number of times?
- Which non-striker has been part of maximum number of runouts?


Finally,

**How to make a generalized IPL query engine?**

In [ ]:
import pandas as pd

In [ ]:
teams_df = pd.read_csv("data/teams.csv")

In [ ]:
matches_df = pd.read_csv("data/matches.csv", index_col='id')

In [ ]:
deliveries_df = pd.read_csv("data/deliveries.csv")

## Which team has taken maximum number of correct toss decisions?

In [ ]:
def get_team_matches(team):
    return len(matches_df.query("team1 == '{0}' or team2 == '{0}'".format(team)))

In [ ]:
teams_df['matches'] = teams_df['name'].apply(get_team_matches)

In [ ]:
toss_match_winner = matches_df.query("toss_winner == winner")['winner'].value_counts()

In [ ]:
teams_df['toss_match_wins'] = teams_df['name'].apply(toss_match_winner.get)

In [ ]:
teams_df.sort_values('toss_match_wins', ascending=False)

In [ ]:
teams_df['correct_toss_perc'] = teams_df['toss_match_wins'] / teams_df['matches']

In [ ]:
teams_df.sort_values('correct_toss_perc', ascending=False)

## Which team has won/lost maximum number of matches while chasing?

In [ ]:
def set_first_innings(match):
    if (match.team1 == match.toss_winner and match.toss_decision == "bat") \
    or (match.team1 != match.toss_winner and match.toss_decision == "field"):
        return match.team1
    else:
        return match.team2
    
    
def set_second_innings(match):
    if (match.team1 == match.toss_winner and match.toss_decision == "field") \
    or (match.team1 != match.toss_winner and match.toss_decision == "bat"):
        return match.team1
    else:
        return match.team2

In [ ]:
matches_df['inning1'] = matches_df.apply(set_first_innings, axis=1)
matches_df['inning2'] = matches_df.apply(set_second_innings, axis=1)

In [ ]:
# maximum number of matches won while chasing
matches_df.query("winner == inning2")['inning2'].value_counts()

In [ ]:
# maximum number of matches lost while chasing
matches_df.query("winner != inning2")['inning2'].value_counts()

## Which toss decision is best for each ground?

In [ ]:
%matplotlib inline

In [ ]:
matches_df.query("toss_winner == winner").groupby('venue')['toss_decision'].value_counts().unstack(level=1).plot.bar(figsize=(15,5))

## Which player has won maximum number of MoM awards while chasing?

In [ ]:
matches_df.query("winner == inning2")['player_of_match'].value_counts().sort_values(ascending=False).head()

## Which batsman has played maximum number of dot balls?

In [ ]:
deliveries_df.query("batsman_runs == 0 and extra_runs == 0")['batsman'].value_counts().head()

## Which batsmen came to crease but never got a chance to face a ball?

In [ ]:
a = deliveries_df['non_striker'].value_counts().index

In [ ]:
b = deliveries_df['batsman'].value_counts().index

In [ ]:
a.difference(b)

## Which bowler has hit for maximum number of 1s/2s/3s/4s/6s?

In [ ]:
deliveries_df.query("batsman_runs == 6")['batsman'].value_counts().head()

## Which batsman got stumped out maximum number of times?

In [ ]:
deliveries_df.query("dismissal_kind == 'stumped'")['batsman'].value_counts().head()

## Which non-striker has been part of maximum number of runouts?

In [ ]:
deliveries_df.query("dismissal_kind == 'run out'")['non_striker'].value_counts().head()


---

# A generalized query engine...

[IPLQE](https://iplqe.herokuapp.com)

**Actors are:**
- Batsmen (batsman)
- Bowlers (bowler)
- Fielders (fielders)
- Non strikers (non_striker)
- Dismissed batsmen (player_out)
- Batting teams (bat_team)
- Bowling teams (bowl_team)

**To calculate parameters:**
- runs (total_runs)
- runs by bat (batsman_runs)
- extra runs (extra_runs)
- wickets (player_out)
- deliveries (ball)

**Filters available:**
- season
- innings
- bat_team
- bowl_team
- over
- ball
- batsman_runs
- extra_runs
- total_runs
- extra_type
- wicket_kind


![](https://i.imgur.com/mernK4C.png)

In [ ]:
filters = "inning == 2 and over == 20"

In [ ]:
actor = "batsman"

In [ ]:
to_calculate = "ball"

In [ ]:
deliveries_df.query(filters).groupby(actor)[to_calculate].count().sort_values(ascending=False).head(10)